In [63]:
import time

import os, sys
import pandas as pd

In [7]:
__scrobbles__ = 'downloads/gps56-2.csv'

In [8]:
pd.options.mode.chained_assignment = None

In [9]:
weekday_map = {0: 'monday',
              1: 'tuesday', 
              2: 'wednesday',
              3: 'thursday', 
              4: 'friday', 
              5: 'saturday', 
              6: 'sunday'}

In [10]:
df = pd.read_csv(__scrobbles__, header = None, names = ['artist', 'album', 'song', 'datetime'])

In [11]:
# remove where datetime is NaN (negligible)
df = df[~df.datetime.isna()]

In [12]:
# convert datetime column to acutal datetime objects
# subtract 7 hours since original times are UTC
df.datetime = pd.to_datetime(df.datetime) - pd.Timedelta(hours = 7)

In [13]:
df.tail(20)

,artist,album,song,datetime
55538,Getter,Wat The Frick EP,Something New,2017-07-05 06:37:00
55539,Che Ecru,buries,2 AM,2017-07-05 06:34:00
55540,Deorro,Good Evening,Guide Me,2017-07-05 06:31:00
55541,BONJR,Hey Sego,hey sego,2017-07-05 06:28:00
55542,Oshi,oshi,state of flux,2017-07-05 06:24:00
55543,Young Thug,Slime Season 3,With Them,2017-07-05 06:20:00
55544,Hyper Potions,Adventures,Adventures,2017-07-04 20:53:00
55545,Young Thug,Beautiful Thugger Girls,Daddy's Birthday,2017-07-04 13:05:00
55546,Young Thug,Slime Season,Again,2017-07-04 13:01:00
55547,Young Thug,Slime Season,Overdosin,2017-07-04 12:50:00


# Time Analysis

In [32]:
df.datetime

1       2020-05-04 17:34:00
2       2020-05-04 15:07:00
3       2020-05-04 15:05:00
4       2020-05-04 15:05:00
5       2020-05-04 14:58:00
                ...        
55553   2017-07-04 12:26:00
55554   2017-07-04 12:23:00
55555   2017-07-01 14:33:00
55556   2017-06-29 20:39:00
55557   2017-06-29 20:34:00
Name: datetime, Length: 55281, dtype: datetime64[ns]

In [42]:
df.datetime.dt.weekday.map(weekday_map).value_counts()

thursday     9829
friday       9606
tuesday      9445
wednesday    7775
monday       7201
saturday     7029
sunday       4396
Name: datetime, dtype: int64

# Lyric Sentiment Analysis

In [44]:
# for genius API
# name           pysent
# URL            https://github.com/LeChase/spotify_hist
# client ID      lT9Ki6NGuPJC8bVLWtmNn0F2J5kYPiL0E7i9pHN8jHot6pnYb_x7rlFlbYAZepEV
# client secret  bElRrQIVRVRZRA3h3iPvUm2iSk6mpeQzYU8H_GPMsMAAJWgSHgSF2j_sigwjJVrvhdJ8d-StiZISXxNKD5cbog
# access token   UgvKlj9wHavgvBZScdTzfIKL2DKiIatIwPVV0M6JpEqm8YgIC_eJiQsYcsnmU2G-

In [14]:
import lyricsgenius, json

In [69]:
genius = lyricsgenius.Genius("UgvKlj9wHavgvBZScdTzfIKL2DKiIatIwPVV0M6JpEqm8YgIC_eJiQsYcsnmU2G-", timeout = 60, sleep_time = 2)

In [71]:
genius.verbose = False

In [17]:
# artist = genius.search_artist("Drake", max_songs=2,)

In [18]:
# artist.save_lyrics()

In [19]:
# # Reading the json as a dict
# with open('Lyrics_Drake.json') as json_data:
#     data = json.load(json_data)

In [20]:
# len(data['songs'])

In [22]:
# song = genius.search_song("somethign new getter")

In [56]:
def get_lyrics(row):
    try:
        song = genius.search_song(' - '.join((row.artist, row.song)))
        return song.lyrics
    except AttributeError:
        return 'NOT FOUND'

In [26]:
test = df.sample(20)

In [116]:
test.lyrics = test.apply(get_lyrics, axis = 1)

Searching for "Still Here Drake"...
Done.
Searching for "Life is Good (feat. Drake) Future"...
Done.
Searching for "Big Amount 2 Chainz"...
Done.
Searching for "Fuck That Check Up (feat. Lil Uzi Vert) Meek Mill"...
Done.
Searching for "Shelter Porter Robinson"...
Done.
Searching for "Rebound aftertheparty"...
Done.
Searching for "Check the Rhime A Tribe Called Quest"...
Done.
Searching for "No Time (feat. Gunna) Playboi Carti"...
Done.
Searching for "Find My Way DaBaby"...
Done.
Searching for "Paramedic! SOB x RBE"...
Done.
Searching for "Pursuit of Happiness (Nightmare) Kid Cudi"...
Done.
Searching for "200 Future"...
Done.
Searching for "Armed and Dangerous Juice WRLD"...
Done.
Searching for "South Galimatias"...
Done.
Searching for "No Role Modelz J. Cole"...
Done.
Searching for "BoomTrap Protocol Logic"...
Done.
Searching for "No Heart 21 Savage"...
Done.
Searching for "Feed Me Dope Future"...
Done.
Searching for "Lil Arrogant (feat. Joey Bada$$ & Russ) IDK"...
Done.
Searching for 

/Users/georgeschnaars/Desktop/Python/spotify_hist/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [118]:
test.lyrics

30513    [Intro]\nDoin' well dog\nYeah, me and all my\n...
7776     [Перевод песни "Future - Life Is Good (feat. D...
14649    [Intro: 2 Chainz]\nI'ma tell you, I'ma tell yo...
54424    [Intro: Meek Mill]\nLil' fish, yeah\nIf you ca...
7591     [Intro: Amy Millan]\nAyy, I will, ayy\nAyy, I ...
53000    [Pre-Chorus]\nBaby I know you want someone\nTo...
16181    [Intro]\nUh, uh, uh, uh\nUh, uh, uh, check the...
26555    [Intro: Playboi Carti]\nCheck it out\nWake up ...
551      [Intro]\nWait a minute, who are you? (Ayy, yo,...
36624    [Intro: Zacari & Kendrick Lamar]\nI am Killmon...
14215    [Intro: Kid Cudi]\nMmmm, ay-oh\nHey...\nRatata...
6283     [Intro: Young Thug]\nEverything right\nEveryth...
21353              \n            Lyrics for this song h...
41499    [Verse 1]\nLook me in the eye I'ma take it slo...
48604    [Produced by Phonix Beats and J. Cole]\n\n[Ver...
44165    [Verse 1]\nLet me get up in this bitch, like, ...
45620    [Intro]\nSouthside, Southside on the, Southsid.

In [121]:
test['lyrics'] = test.lyrics

In [122]:
test

,artist,album,song,datetime,lyrics
30513,Drake,Views,Still Here,2019-01-29 00:58:00,"[Intro]\nDoin' well dog\nYeah, me and all my\n..."
7776,Future,Life Is Good (feat. Drake),Life is Good (feat. Drake),2020-01-10 04:00:00,"[Перевод песни ""Future - Life Is Good (feat. D..."
14649,2 Chainz,Pretty Girls Like Trap Music,Big Amount,2019-10-10 13:41:00,"[Intro: 2 Chainz]\nI'ma tell you, I'ma tell yo..."
54424,Meek Mill,Wins & Losses,Fuck That Check Up (feat. Lil Uzi Vert),2017-07-25 05:29:00,"[Intro: Meek Mill]\nLil' fish, yeah\nIf you ca..."
7591,Porter Robinson,Shelter,Shelter,2020-01-13 06:15:00,"[Intro: Amy Millan]\nAyy, I will, ayy\nAyy, I ..."
53000,aftertheparty,?!,Rebound,2017-08-14 07:50:00,[Pre-Chorus]\nBaby I know you want someone\nTo...
16181,A Tribe Called Quest,The Low End Theory,Check the Rhime,2019-09-21 13:16:00,"[Intro]\nUh, uh, uh, uh\nUh, uh, uh, check the..."
26555,Playboi Carti,Die Lit,No Time (feat. Gunna),2019-03-29 13:57:00,[Intro: Playboi Carti]\nCheck it out\nWake up ...
551,DaBaby,Find My Way,Find My Way,2020-04-24 04:53:00,"[Intro]\nWait a minute, who are you? (Ayy, yo,..."
36624,SOB x RBE,Black Panther the Album Music From and Inspire...,Paramedic!,2018-10-04 08:30:00,[Intro: Zacari & Kendrick Lamar]\nI am Killmon...


In [157]:
%%timeit -r1
get_lyrics(test.iloc[0])

Searching for "Drake Still Here"...
Done.
Searching for "Drake Still Here"...
Done.
2.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [34]:
test_lyrics = get_lyrics(test.iloc[0])

In [35]:
len(test_lyrics)

3966

In [167]:
sys.getsizeof(test_lyrics)

4180

In [169]:
test['bytes'] = test.lyrics.apply(sys.getsizeof)

In [174]:
print(test.bytes.mean()*len(dupes)/1000000, 'MB total')

28.959085599999998 MB total


In [178]:
print(2.2 * len(dupes)/60/60, 'hrs to scrape')

4.6096111111111115 hrs to scrape


In [1]:
import util

In [2]:
help(util)

Help on module util:

NAME
    util - util module

FUNCTIONS
    suppress_stdout(func)
        decorator to redirect print output to null

FILE
    /Users/georgeschnaars/Desktop/Python/spotify_hist/util.py




In [3]:
help(util.suppress_stdout)

Help on function suppress_stdout in module util:

suppress_stdout(func)
    decorator to redirect print output to null



In [44]:
uniques = df.drop_duplicates(['artist', 'song'])[['artist', 'song']]

In [72]:
print('starting:', time.ctime())
uniques['lyrics'] = uniques.apply(get_lyrics, axis = 1)
print('ending:', time.ctime())

starting: Mon May  4 21:57:10 2020


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [110]:
test = uniques.groupby('artist')['song'].apply(list).reset_index(name = 'songs')

In [234]:
test

,artist,songs
0,!!!,[Even When the Water's Cold]
1,$uicideboy$,"[Either Hated Or Ignored, PARIS, REIGN IN BLOO..."
2,03 Greedo,"[Maria, Disco Shit, Beg Your Pardon, Soul Food..."
3,070 Shake,[Honey]
4,1-800 GIRLS,[All Day All Night]
...,...,...
2039,wasiu,[Cigarettes & Poutine]
2040,yugi boi,"[Peach Tea, The End, No Problem]"
2041,yuzu blur,[Sky Blush]
2042,ÊMIA,[Psychic]


In [235]:
d = test.to_dict('records')

In [237]:
# d

In [130]:
d = {item['artist']: item['songs'] for item in d}

In [132]:
len(d)

2044

In [136]:
# for artist, songs in d.items():
#     for song in songs:
#         print(artist, song)

In [137]:
import json

In [143]:
with open('uniques.json', 'w') as f:
    json.dump(d, f)

In [224]:
import collections

In [144]:
with open('uniques.json') as f:
    my_dict = json.load(f)

In [147]:
my_dict['TEST'] = ['hello']

In [179]:
my_dict['Drake'].append('tes5')

In [186]:
def dict_diff(dict_old, dict_new):
    # returns the items present in dict_b that are NOT present in dict_a
    return set(flatten_dict(dict_new)) - set(flatten_dict(dict_old))

In [190]:
diff = dict_diff(d, my_dict)

In [208]:
diff

{('Drake', 'tes5'), ('Drake', 'test'), ('TEST', 'hello')}

In [217]:
d = defaultdict(list)

In [218]:
for artist, song in diff:
    d[artist].append(song)

In [228]:
unflatten_dict(flatten_dict(d))

defaultdict(list, {'TEST': ['hello'], 'Drake': ['test', 'tes5']})

In [220]:
dict(d)

{'TEST': ['hello'], 'Drake': ['test', 'tes5']}

In [230]:
def flatten_dict(d):
    # ungroups keys
    return [(key, value) for key, values in d.items() for value in values]

In [229]:
def unflatten_dict(list_tups):
    # groups keys
    d = collections.defaultdict(list)
    for key, val in list_tups:
        d[key].append(val)
    return d